In [1]:
import pyodbc
import requests

DBServerName = 'applinkedin.database.windows.net'

conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=applinkedin.database.windows.net;' #your server name
                      'Database=-------------;'                  #your DB
                      'UID=-------------;'                     #your user       
                      'PWD=-------------')                           #your password
cursor = conn.cursor()


## Get request and output as json for manipulation

In [2]:
url = 'https://api.uplead.com/v2/company-search?company=amazon'
headers = {'Authorization': '-------------'} #use your API key
out = requests.request('GET',url,headers=headers).json()


##  See json dictionary keys 

In [3]:
for i in out:
    print(i)

data
userInfo


## See dictionary keys inside 'data'

In [4]:
for i in out['data']:
    print(i)

id
company_name
address
city
state
county
country
zip
phone_number
fax_number
revenue
industry
naics_description
description
naics_code
domain
employees
linkedin_url
twitter_url
facebook_url
youtube_url
crunchbase_url
yelp_url
instagram_url
type
ticker
exchange
alexa_rank
year_founded
sic_code
sic_description
logo


## Assigning data as out['data'] for readibility and populating variables with its content

In [5]:
data = out['data']
street = data['address']
state = data['state']
zip = data['zip']

print(street, state,zip)

410 Terry Ave N WA 98109-5210


## Code works, time to adjust it and follow the plan. 
## The application must retrieve company names from the ApplicationDB and use those names on the API to get information about street, state, and zip. 
### For testing purposes, let's see what is inside the Company table first

In [10]:
before_select = 'select * from company'
before_out = cursor.execute(before_select)
for i in before_out: 
    print(i)

(1, 'Microsoft', None, None, None)
(2, 'Apple', None, None, None)
(3, 'Amazon', None, None, None)


C:\Users\Gui\AppData\Local\Temp\ipykernel_31124\3064811835.py:4: DeprecationWarning: PyUnicode_FromUnicode(NULL, size) is deprecated; use PyUnicode_New() instead
  print(i)


### Microsoft, Apple, Amazon have None values for street, state, and zip

### First step is to retrieve info from DB. name_list is a list comprehension that simulates a for loop and creates a list with the results

In [11]:
select_statement = 'select Name from Company'
db_out = cursor.execute(select_statement)
name_list = [i[0] for i in db_out]
print(name_list)

['Amazon', 'Apple', 'Microsoft']


### Repeat code from before but substitute URL for Company name. Then, update the missing values on the DB with new information. 

    

In [12]:
for i in name_list:
    url = f'https://api.uplead.com/v2/company-search?company={i}'
    headers = {'Authorization': 'fa79bda7ebe6cf411046980814a0d06e'}
    out = requests.request('GET',url,headers=headers).json()
    data = out['data']
    street = data['address']
    state = data['state']
    zip = data['zip']
    print(street,state,zip)
    update = 'update Company set Street = (?) ,State =(?) ,Zip =(?) where Name = (?)'
    cursor.execute(update,street,state,zip,i)
    cursor.commit()

410 Terry Ave N WA 98109-5210
1 Apple Park Way CA 95014-0642
39 Quai Du Président Roosevelt None 92130


In [13]:
after_update = 'select * from company'
after_out = cursor.execute(after_update)
for i in after_out:
    print(i)

(1, 'Microsoft', '39 Quai Du Président Roosevelt', None, '92130')
(2, 'Apple', '1 Apple Park Way', 'CA', '95014-0642')
(3, 'Amazon', '410 Terry Ave N', 'WA', '98109-5210')


C:\Users\Gui\AppData\Local\Temp\ipykernel_31124\562489209.py:4: DeprecationWarning: PyUnicode_FromUnicode(NULL, size) is deprecated; use PyUnicode_New() instead
  print(i)


### All works. The script takes information from the Database Company table and uses those company names as an argument for UpLead API. It retrieves information for street, state, and zip and updates the values on the database. 

# Final Notes & Improvements
## This script is supposed to automate the process of filling the information, therefore, it should be placed in a cloud environment where it can run every X minutes
## One major improvement is to write a better select query. For now, it always selects all companies in the table. In the future, it should only select the companies where address, state, and zip = null. 
## That saves computing power and we work with smaller datasets. It only captures companies who don't have address information instead of repeating all of them again. 